In [1]:
import json
import pandas as pd
import numpy as np
agents = 2
num_agents = 2
items = 3
type_of_dist = 'exponential'

# valuation table
#    0   1   2
# 0  69  43  63
# 1  84  93  48

valuations = np.array([[69, 43, 63], [84, 93, 48]])



In [10]:
paths = ['/Users/satyamgoyal/Downloads/Vs Code/llm-fairness/outputs/agents_2/items_3/gpt4om/zero_shot0/output_1.txt', '/Users/satyamgoyal/Downloads/Vs Code/llm-fairness/outputs/agents_2/items_3/gpt4om/zero_shot0/output_1.txt']
from model_response import model_init, query_model, extract_json
from valuation_generation import generate_valuations

import numpy as np
import pandas as pd

agents = 2
items = 3

allocation_matrices = np.zeros((2, agents, items))

valuation_tables = np.zeros((2, agents, items))

for i in range(2):
    table = pd.DataFrame(generate_valuations(agents, items, scale=10))
    # valuations = table.values
    valuation_tables[i] = valuations

    allocation = extract_json(f"{paths[i]}")
    print(allocation)
    for j in range(agents):
            for item in allocation[j]:
                allocation_matrices[i][j][item] = 1

print(allocation_matrices)
print(valuation_tables)
allocation_matrices_T = np.transpose(allocation_matrices, (0, 2, 1))

allocation_table = np.matmul(valuation_tables, allocation_matrices_T)

envy_matrix_clipped = np.maximum(0, allocation_table - np.diagonal(allocation_table, axis1=1, axis2=2)[:, :, np.newaxis])
envy_matrix_unclipped = allocation_table - np.diagonal(allocation_table, axis1=1, axis2=2)[:, :, np.newaxis]

n, m, _ = envy_matrix_unclipped.shape
i, j = np.arange(m), np.arange(m)

# Use advanced indexing to set the diagonals
envy_matrix_unclipped[:, i, j] = -np.inf

print(envy_matrix_clipped)
print(envy_matrix_unclipped)
# verify
# for i in range(2):
#     alloc = valuation_tables[i] @ allocation_matrices[i].T
#     print(np.maximum(0, alloc - np.diag(alloc)[:, np.newaxis]))

# print(envy_matrix_clipped)

{0: [2], 1: [0, 1], 2: []}
{0: [2], 1: [0, 1], 2: []}
[[[0. 0. 1.]
  [1. 1. 0.]]

 [[0. 0. 1.]
  [1. 1. 0.]]]
[[[69. 43. 63.]
  [84. 93. 48.]]

 [[69. 43. 63.]
  [84. 93. 48.]]]
[[[ 0. 49.]
  [ 0.  0.]]

 [[ 0. 49.]
  [ 0.  0.]]]
[[[ -inf   49.]
  [-129.  -inf]]

 [[ -inf   49.]
  [-129.  -inf]]]


In [12]:
# print(envy_matrix_clipped[0].max(axis=1))
# print(envy_matrix_clipped[1].max(axis=1))
# envy_matrix_clipped.max(axis=2)
max_envy = envy_matrix_clipped.max(axis=2)
max_envy_unclipped = envy_matrix_unclipped.max(axis=1)

sum_envy = envy_matrix_clipped.sum(axis=2)
sum_envy_unclipped = envy_matrix_unclipped.sum(axis=2)

print(max_envy)
print(max_envy_unclipped)


# test_id = [1,2]
# distribution = 'uniform'
# num_agents = agents
# num_items = items



# envy_freeness = pd.DataFrame()

# envy_freeness['Test_Id'] = range(1, 3)

# for i in range(num_agents):
#     envy_freeness['Agent_' + str(i+1) + '_Max'] = max_envy[:][i]
#     envy_freeness['Agent_' + str(i+1) + '_Max_Unclipped'] = max_envy_unclipped[:][i]
#     envy_freeness['Agent_' + str(i+1) + '_Sum'] = sum_envy[:][i]
#     envy_freeness['Agent_' + str(i+1) + '_Sum_Unclipped'] = sum_envy_unclipped[:][i]


# envy_freeness['Max_Envies'] = max_envy.max()
# envy_freeness['Max_Envies_Unclipped'] = max_envy_unclipped.max()
# envy_freeness['Sum_Envies'] = sum_envy.sum()
# envy_freeness['Sum_Envies_Unclipped'] = sum_envy_unclipped.sum()
# envy_freeness['Envy_Freeness'] = 1 if max_envy.max() == 0 else 0

# envy_freeness.to_csv('envy_freeness2.csv', index=False)

[[49.  0.]
 [49.  0.]]
[[-129.   49.]
 [-129.   49.]]


In [13]:
from envy_freeness import calculate_envy

envy_matrix_c, envy_matrix_u  = calculate_envy(valuations, allocation, num_agents=num_agents, num_items=items)

Allocation Table:  [[ 5.09  6.57]
 [26.58 30.43]]
Allocation Matrix:  [[0. 1. 0.]
 [1. 0. 1.]]
Allocation Table:  [[ 5.09  6.57]
 [26.58 30.43]]


In [18]:

# get the max and sum of every column
max_envy = envy_matrix_c.max(axis=1)
max_envy_unclipped = envy_matrix_u.max(axis=1)

sum_envy = envy_matrix_c.sum(axis=1)
sum_envy_unclipped = envy_matrix_u.sum(axis=1)

test_id = 1
distribution = type_of_dist
num_agents = agents
num_items = items
path = f"{agents}_{items}"

envy_freeness = pd.DataFrame()

envy_freeness['Test_Id'] = [test_id]
for i in range(num_agents):
    envy_freeness['Agent_' + str(i+1) + '_Max'] = max_envy[i]
    envy_freeness['Agent_' + str(i+1) + '_Max_Unclipped'] = max_envy_unclipped[i]
    envy_freeness['Agent_' + str(i+1) + '_Sum'] = sum_envy[i]
    envy_freeness['Agent_' + str(i+1) + '_Sum_Unclipped'] = sum_envy_unclipped[i]


envy_freeness['Max_Envies'] = max_envy.max()
envy_freeness['Max_Envies_Unclipped'] = max_envy_unclipped.max()
envy_freeness['Sum_Envies'] = sum_envy.sum()
envy_freeness['Sum_Envies_Unclipped'] = sum_envy_unclipped.sum()
envy_freeness['Envy_Freeness'] = 1 if max_envy.max() == 0 else 0

envy_freeness.to_csv('envy_freeness.csv', index=False)

SyntaxError: invalid syntax (3987016503.py, line 12)